In [1]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [2]:
import torch

In [3]:
from datasets import load_dataset

dataset = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [5]:
dataset["train"][0]
# Output: {'translation': {'en': '...', 'hi': '...'}}


{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [7]:
tokenizer.add_special_tokens({
    'additional_special_tokens': ['<sos>', '<eos>']
})


2

In [8]:
sos_token_id = tokenizer.convert_tokens_to_ids('<sos>')
eos_token_id = tokenizer.convert_tokens_to_ids('<eos>')
pad_token_id = tokenizer.pad_token_id  # Already built-in


In [9]:
source_texts = ["I love programming", "How are you?"]       # English
target_texts = ["मुझे प्रोग्रामिंग पसंद है", "आप कैसे हैं?"]  # Hindi

def encode_pair(src, tgt):
    src_enc = tokenizer(src, padding="max_length", truncation=True, max_length=32, return_tensors="pt")

    tgt_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tgt))
    tgt_ids = [sos_token_id] + tgt_ids + [eos_token_id]
    tgt_ids = tgt_ids[:32]
    tgt_ids += [pad_token_id] * (32 - len(tgt_ids))

    return src_enc['input_ids'].squeeze(), torch.tensor(tgt_ids)


In [10]:
import torch
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
  def __init__(self, source_texts, target_texts, tokenizer, max_len = 32):
    self.source_texts = source_texts
    self.target_texts = target_texts
    self.tokenizer = tokenizer
    self.max_len = max_len

    self.sos_token_id = tokenizer.convert_tokens_to_ids("<sos>")
    self.eos_token_id = tokenizer.convert_tokens_to_ids("<eos>")
    self.pad_token_id = tokenizer.pad_token_id

  def __len__(self):
    return len(self.source_texts)

  def __getitem__(self, idx):
    src = self.source_texts[idx]
    tgt = self.target_texts[idx]

    # Tokenize source
    src_enc = self.tokenizer(
        src, padding="max_length", truncation=True,
        max_length=self.max_len, return_tensors="pt"
    )['input_ids'].squeeze()

    # Tokenize and prepare target
    tgt_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(tgt))
    tgt_tokens = tgt_tokens[:self.max_len - 2]

    tgt_input = [self.sos_token_id] + tgt_tokens
    tgt_output = tgt_tokens + [self.eos_token_id]

    # Pad both
    tgt_input += [self.pad_token_id] * (self.max_len - len(tgt_input))
    tgt_output += [self.pad_token_id] * (self.max_len - len(tgt_output))

    return {
        "src_input": torch.tensor(src_enc, dtype=torch.long),
        "tgt_input": torch.tensor(tgt_input, dtype=torch.long),
        "tgt_output": torch.tensor(tgt_output, dtype=torch.long),
    }

In [11]:
from torch.utils.data import DataLoader

dataset = TranslationDataset(source_texts, target_texts, tokenizer, max_len=32)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    print(batch['src_input'].shape)   # [32, 32]
    print(batch['tgt_input'].shape)   # [32, 32]
    print(batch['tgt_output'].shape)  # [32, 32]
    break


torch.Size([2, 32])
torch.Size([2, 32])
torch.Size([2, 32])


/tmp/ipython-input-10-3113381225.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "src_input": torch.tensor(src_enc, dtype=torch.long),


In [12]:
import torch.nn as nn

class Encoder(nn.Module):
  def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers = 1, dropout = 0.1):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx = 0)
    self.gru = nn.GRU(emb_dim, hidden_dim, num_layers, batch_first = True)

  def forward(self, src):
    embedded = self.embedding(src)
    outputs, hidden = self.gru(embedded)

    return hidden

In [13]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers = 1, dropout = 0.1):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx = 0)
    self.gru = nn.GRU(emb_dim, hidden_dim, num_layers, batch_first = True)
    self.fc_out = nn.Linear(hidden_dim, vocab_size)

  def forward(self, input_token, hidden):
    input_token = input_token.unsqueeze(1)
    embedded = self.embedding(input_token)

    output, hidden = self.gru(embedded, hidden)
    prediction = self.fc_out(output.squeeze(1))

    return prediction, hidden

In [14]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, pad_token_id, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device
    self.pad_token_id = pad_token_id


  def forward(self, src, tgt_input, teacher_forcing_ratio = 0.5):
    batch_size, tgt_len = tgt_input.shape
    vocab_size = self.decoder.fc_out.out_features

    outputs = torch.zeros(batch_size, tgt_len, vocab_size).to(self.device)
    hidden = self.encoder(src)

    input_token = tgt_input[:, 0]

    for t in range(1, tgt_len):
      output, hidden = self.decoder(input_token, hidden)
      outputs[:, t] = output

      teacher_force = torch.rand(1).item() < teacher_forcing_ratio
      top1 = output.argmax(1)

      input_token = tgt_input[:, t] if teacher_force else top1

    return outputs

In [15]:
# Get total vocab size after adding special tokens
vocab_size = tokenizer.vocab_size + len(tokenizer.additional_special_tokens)

# Set model hyperparameters
embedding_dim = 256
hidden_dim = 512
num_layers = 1

# Get special token IDs
pad_token_id = tokenizer.pad_token_id
sos_token_id = tokenizer.convert_tokens_to_ids('<sos>')
eos_token_id = tokenizer.convert_tokens_to_ids('<eos>')
emb_dim = 256
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [16]:
encoder = Encoder(
    vocab_size=vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
)

decoder = Decoder(
    vocab_size=vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
)
model = Seq2Seq(
    encoder=encoder,
    decoder=decoder,
    pad_token_id=pad_token_id,
    device=device
).to(device)


In [17]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [18]:
def train(model, dataloader, optimizer, criterion, device, clip=1.0):
    model.train()
    epoch_loss = 0

    for batch in dataloader:
        src = batch['src_input'].to(device)         # [B, src_len]
        tgt_input = batch['tgt_input'].to(device)   # [B, tgt_len]
        tgt_output = batch['tgt_output'].to(device) # [B, tgt_len]

        optimizer.zero_grad()

        output = model(src, tgt_input)              # [B, tgt_len, vocab_size]
        output_dim = output.shape[-1]

        # Flatten for loss computation
        output = output[:, 1:].reshape(-1, output_dim)       # skip first timestep
        tgt_output = tgt_output[:, 1:].reshape(-1)

        loss = criterion(output, tgt_output)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


In [19]:
dataset = TranslationDataset(source_texts, target_texts, tokenizer, max_len=32)
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset,
    batch_size=32,      # you can adjust this
    shuffle=True,       # shuffle each epoch
    drop_last=True      # ensures all batches are full-sized
)


In [20]:
for epoch in range(100):
    train_loss = train(model, dataloader, optimizer, criterion, device)
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f}")


/tmp/ipython-input-10-3113381225.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "src_input": torch.tensor(src_enc, dtype=torch.long),


Epoch 1 | Train Loss: 11.6634
Epoch 2 | Train Loss: 11.4997
Epoch 3 | Train Loss: 11.3613
Epoch 4 | Train Loss: 10.9638
Epoch 5 | Train Loss: 10.9045
Epoch 6 | Train Loss: 10.4613
Epoch 7 | Train Loss: 9.1068
Epoch 8 | Train Loss: 8.4520
Epoch 9 | Train Loss: 7.2600
Epoch 10 | Train Loss: 6.4118
Epoch 11 | Train Loss: 5.4320
Epoch 12 | Train Loss: 4.8896
Epoch 13 | Train Loss: 3.9007
Epoch 14 | Train Loss: 3.1927
Epoch 15 | Train Loss: 2.6732
Epoch 16 | Train Loss: 2.0970
Epoch 17 | Train Loss: 1.9973
Epoch 18 | Train Loss: 1.6947
Epoch 19 | Train Loss: 1.5068
Epoch 20 | Train Loss: 1.4772
Epoch 21 | Train Loss: 1.3652
Epoch 22 | Train Loss: 1.7279
Epoch 23 | Train Loss: 1.1348
Epoch 24 | Train Loss: 0.9298
Epoch 25 | Train Loss: 1.0618
Epoch 26 | Train Loss: 1.4386
Epoch 27 | Train Loss: 1.1142
Epoch 28 | Train Loss: 1.0869
Epoch 29 | Train Loss: 0.9278
Epoch 30 | Train Loss: 1.4498
Epoch 31 | Train Loss: 0.7900
Epoch 32 | Train Loss: 0.8854
Epoch 33 | Train Loss: 0.8785
Epoch 34 | Tr

In [21]:
def translate_sentence(model, sentence, tokenizer, max_len=32, device="cpu"):
    model.eval()

    # Tokenize input → returns shape [1, seq_len]
    tokens = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=max_len)
    src_input = tokens['input_ids'].to(device)  # shape = [1, seq_len]

    # Encode
    with torch.no_grad():
        hidden = model.encoder(src_input)  # ✅ expects [batch, seq_len]

    # Prepare for decoding
    sos_token_id = tokenizer.convert_tokens_to_ids("<sos>")
    eos_token_id = tokenizer.convert_tokens_to_ids("<eos>")

    input_token = torch.tensor([sos_token_id], device=device)  # shape = [1]
    output_tokens = []

    for _ in range(max_len):
        with torch.no_grad():
            output, hidden = model.decoder(input_token, hidden)
            top1 = output.argmax(1).item()

        if top1 == eos_token_id:
            break

        output_tokens.append(top1)
        input_token = torch.tensor([top1], device=device)

    # Convert token IDs to text
    translated_tokens = tokenizer.convert_ids_to_tokens(output_tokens)
    translated_text = tokenizer.convert_tokens_to_string(translated_tokens)

    return translated_text


In [22]:
english_sentence = "I love coding"
translated = translate_sentence(model, english_sentence, tokenizer, device=device)
print("Predicted Hindi:", translated)


Predicted Hindi: ##ुझे प्रोग्रामिंग पसंद
